Please modify SweepingFCNN.ipynb and fcnn_models.py so I can do a small sweep of models to try.
I want you to add 6 more models in fcnn_models.py (BirdFCNN_v2, ..., BirdFCNN_v7) with variations in the architectures or training parameters targeted at optimizing audio classification based on 70,112-pixel vectors of grayscale log-mel spectrograms with 2400 training samples. Make sure they are all called iN SweepingFCNN.ipynb, where the single-fold train-val is run, and after running all of them, plot for each model a 2x2 plot of subplots:
- train & val loss (cross-entropy) curves
- best f1 score
- best accuracy
- confusion matrix

For the plotting part, you can use plot_full_metrics from metrics.py, ensuring that the format the models save their training data matches what the plot function needs. **DO NOT MODIFY metrics.py OR ANY FILE IN utils/ OTHER THAN fcnn_models.py! ONLY MODIFY fcnn_models.py and SweepingFCNN.ipynb**.